# Connecting and publishing to Google pubsub

In [2]:
import os
import base64
import json
import ndjson
from google.cloud import pubsub_v1
from google.oauth2 import service_account

In [3]:
project = 'kind-lab' # 'kind-lab

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project, 'mimic-fhir-bundles')
topic_path

'projects/kind-lab/topics/mimic-fhir-bundles'

In [4]:
# auth = os.popen("gcloud auth application-default print-access-token").read().strip()
blob_dir='bundle-loading/custom-folder'

org_bundle=b'{"resourceType": "Bundle", "id":"org-bundle","type":"transaction","entry":[{"fullUrl": "ee172322-118b-5716-abbc-18e4c5437e15", "resource":{"id": "ee172322-118b-5716-abbc-18e4c5437e15", "meta": {"profile": ["http://fhir.mimic.mit.edu/StructureDefinition/mimic-organization"]}, "name": "Beth Israel Deaconess Medical Center", "type": [{"coding": [{"code": "prov", "system": "http://terminology.hl7.org/CodeSystem/organization-type", "display": "Healthcare Provider"}]}], "active": true, "identifier": [{"value": "1194052720", "system": "http://hl7.org/fhir/sid/us-npi"}], "resourceType": "Organization"},"request":{"method":"PUT","url":"Organization/ee172322-118b-5716-abbc-18e4c5437e15"}}]}'
bundle= b'{"resourceType":"Bundle","id":"bundle-transaction","type":"transaction","entry":[{"fullUrl":"ba83e5fd-2a2d-5a24-94cc-a08d1294f3d6","resource":{"id":"ba83e5fd-2a2d-5a24-94cc-a08d1294f3d6","meta":{"profile":["http://fhir.mimic.mit.edu/StructureDefinition/mimic-encounter"]},"type":[{"coding":[{"code":"99218","system":"http://fhir.mimic.mit.edu/CodeSystem/mimic-hcpcs-cd","display":"Hospital observation services"}]},{"coding":[{"code":"36252","system":"http://fhir.mimic.mit.edu/CodeSystem/mimic-hcpcs-cd","display":"Cardiovascular system"}]}],"class":{"code":"AMB","system":"http://terminology.hl7.org/CodeSystem/v3-ActCode","display":"ambulatory"},"period":{"end":"2143-03-19T12:00:00-04:00","start":"2143-03-18T12:34:00-04:00"},"status":"finished","subject":{"reference":"Patient/adde1635-3110-5e92-b9f0-7a6d845a1784"},"location":[{"period":{"end":"2143-03-19T12:03:31-04:00","start":"2143-03-18T12:34:29-04:00"},"location":{"reference":"Location/25fdf8f3-5c5e-517b-b732-8f4501f25c33"}}],"priority":{"coding":[{"code":"R","system":"http://terminology.hl7.org/CodeSystem/v3-ActPriority","display":"routine"}]},"identifier":[{"use":"usual","value":"27125816","system":"http://fhir.mimic.mit.edu/identifier/encounter-hosp","assigner":{"reference":"Organization/ee172322-118b-5716-abbc-18e4c5437e15"}}],"serviceType":{"coding":[{"code":"CMED","system":"http://fhir.mimic.mit.edu/CodeSystem/mimic-services"}]},"resourceType":"Encounter","hospitalization":{"admitSource":{"coding":[{"code":"PROCEDURE SITE","system":"http://fhir.mimic.mit.edu/CodeSystem/mimic-admit-source"}]}},"serviceProvider":{"reference":"Organization/ee172322-118b-5716-abbc-18e4c5437e15"}},"request":{"method":"PUT","url":"Encounter/ba83e5fd-2a2d-5a24-94cc-a08d1294f3d6"}},{"fullUrl":"1f9c4560-e98f-55ea-815f-0c1a0f7a5ae7","resource":{"id":"1f9c4560-e98f-55ea-815f-0c1a0f7a5ae7","meta":{"profile":["http://fhir.mimic.mit.edu/StructureDefinition/mimic-encounter"]},"type":[{"coding":[{"code":"308335008","system":"http://snomed.info/sct","display":"Patient encounter procedure"}]}],"class":{"code":"AMB","system":"http://terminology.hl7.org/CodeSystem/v3-ActCode","display":"ambulatory"},"period":{"end":"2143-03-10T11:35:00-04:00","start":"2143-03-09T11:10:00-05:00"},"status":"finished","subject":{"reference":"Patient/b410dd44-7d65-56f9-974f-2751e8aa80e2"},"location":[{"period":{"end":"2143-03-09T16:25:33-05:00","start":"2143-03-09T11:11:33-05:00"},"location":{"reference":"Location/f4ed77ed-3be3-5700-bc6a-3622ca10f90f"}},{"period":{"end":"2143-03-10T12:00:01-04:00","start":"2143-03-09T16:25:33-05:00"},"location":{"reference":"Location/25fdf8f3-5c5e-517b-b732-8f4501f25c33"}}],"priority":{"coding":[{"code":"R","system":"http://terminology.hl7.org/CodeSystem/v3-ActPriority","display":"routine"}]},"identifier":[{"use":"usual","value":"21216581","system":"http://fhir.mimic.mit.edu/identifier/encounter-hosp","assigner":{"reference":"Organization/ee172322-118b-5716-abbc-18e4c5437e15"}}],"serviceType":{"coding":[{"code":"CMED","system":"http://fhir.mimic.mit.edu/CodeSystem/mimic-services"}]},"resourceType":"Encounter","hospitalization":{"admitSource":{"coding":[{"code":"PROCEDURE SITE","system":"http://fhir.mimic.mit.edu/CodeSystem/mimic-admit-source"}]}},"serviceProvider":{"reference":"Organization/ee172322-118b-5716-abbc-18e4c5437e15"}},"request":{"method":"PUT","url":"Encounter/1f9c4560-e98f-55ea-815f-0c1a0f7a5ae7"}}]}'

In [5]:
future = publisher.publish(topic_path, org_bundle, blob_dir=blob_dir)
future

<Future at 0x7fb22598a790 state=pending>

In [11]:
bundle_json = json.loads(bundle.decode('utf-8'))
bundle_json.encode()

AttributeError: 'dict' object has no attribute 'encode'

In [59]:
loaded_bundle = json.loads(bundle.decode("utf-8"))
err_bundle = {"bundle": loaded_bundle, "error": 'some error message, really bad stuff'}

In [56]:
from google.cloud import storage

In [62]:
storage_client = storage.Client()
bucket = storage_client.get_bucket('mimic-fhir')
blob = bucket.blob('bundle-loading/error/some-file3.json')
blob.upload_from_string(json.dumps(err_bundle))

In [86]:
# Test out creating a summary error file
error_folder = f'{blob_dir}/error'
local_dir = '/home/alex/Downloads'
summary_err_file = f'{local_dir}/summary_err.ndjson'
client = storage.Client(project='kind-lab')
with open(summary_err_file, 'w') as f:
    writer = ndjson.writer(f, ensure_ascii=False)
    for blob in client.list_blobs('mimic-fhir', prefix=error_folder):
        blob_data = json.loads(blob.download_as_string())
        err_line = {"error": blob_data['error'], "bundle_id": blob_data['bundle']['id']}
        print(err_line)
        writer.writerow(err_line)

blob = bucket.blob(f'{blob_dir}/err_summary.ndson')
blob.upload_from_filename(summary_err_file)




{'error': 'some error', 'bundle_id': 'bundle-transaction'}
{'error': 'some error', 'bundle_id': 'bundle-transaction'}


In [77]:
error_folder

'bundle-loading/custom-folder/errror'

In [90]:
json.loads(org_bundle.decode('utf-8'))

{'resourceType': 'Bundle',
 'id': 'org_bundle',
 'type': 'transaction',
 'entry': [{'fullUrl': 'ee172322-118b-5716-abbc-18e4c5437e15',
   'resource': {'id': 'ee172322-118b-5716-abbc-18e4c5437e15',
    'meta': {'profile': ['http://fhir.mimic.mit.edu/StructureDefinition/mimic-organization']},
    'name': 'Beth Israel Deaconess Medical Center',
    'type': [{'coding': [{'code': 'prov',
        'system': 'http://terminology.hl7.org/CodeSystem/organization-type',
        'display': 'Healthcare Provider'}]}],
    'active': True,
    'identifier': [{'value': '1194052720',
      'system': 'http://hl7.org/fhir/sid/us-npi'}],
    'resourceType': 'Organization'},
   'request': {'method': 'PUT',
    'url': 'Organization/ee172322-118b-5716-abbc-18e4c5437e15'}}]}

In [13]:
from uuid import uuid4

In [15]:
str(uuid4())

'6bf8d00d-c6b0-4318-8f85-0aa95ce06c9a'